In [1]:
import pandas as pd
import numpy as np
import os
import re
import string
import gc
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import xgboost as xgb

In [22]:
train=pd.read_csv('F:/hackthons/Hacker earth/Effectiveness of STD drug/train.csv')
test=pd.read_csv('F:/hackthons/Hacker earth/Effectiveness of STD drug/test.csv')

In [3]:
train.head(3)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969


Adding Sentiment Intensity to Review

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
train = train.join(train['review_by_patient'].apply(lambda x: analyzer.polarity_scores(x)).apply(pd.Series))
test = test.join(test['review_by_patient'].apply(lambda x: analyzer.polarity_scores(x)).apply(pd.Series))

In [24]:
def clean_text(text):
    text=text.lower() 
    text=re.sub(r"\d+","",text) 
    text=text.translate(str.maketrans("","",string.punctuation))  
    text=re.sub('\s+',' ',text)  
    text=[token for token in text.split() if len(token)>2]   
    text=' '.join(text)
    shortword=re.compile(r'\W*\b\w{10,}\b')   
    text=shortword.sub('',text)
    stop_words=set(stopwords.words('english')) 
    tokens=word_tokenize(text)
    ps=PorterStemmer()
    
    text=''.join(text)
    return text

In [25]:
train['Cleaned_Text'] = train['review_by_patient'].apply(clean_text)
test['Cleaned_Text'] = test['review_by_patient'].apply(clean_text)

In [27]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [59]:
stopword = set(stopwords.words('english'))
def convert(words) :
  
    current_words = []
    stop_words=set(stopwords.words('english')) 
    for i in words :
        if i not in stopword :
            
            updated_word = ps.stem(i)
            current_words.append(updated_word.lower())
    return current_words

In [60]:
train['lemma']=train['Cleaned_Text'].apply(convert)
test['lemma']=test['Cleaned_Text'].apply(convert)

In [57]:
train['lemma']

0        [h,  , e,  , e, f, f, e, c,  , k, e,  , b, l, ...
1        [n,  , h, l, f, w,  , h, r, u, g, h,  , h,  , ...
2        [u, e,  , k, e,  , n, h, e, r,  , r, l,  , w, ...
3        [u, b, x, n, e,  , h,  , u, r, n, e,  , l, f, ...
4        [ , r, e,  , w, r, k,  , w, h,  , r, c, k,  , ...
5        [h,  , b, e, e, n,  , h, e,  , p, l, l,  , f, ...
6        [h, v, e,  , k, e, n,  , f, r,  , e, r,  , w, ...
7        [v, e,  , b, e, e, n,  , k, n, g,  , n, c, e, ...
8        [v, e,  , b, e, e, n,  , e, v, e, r,  , e, c, ...
9        [p, r, n, g,  , w,  , w, h,  , p, n, u, e, n, ...
10       [l, v, e,  , w, e, e, r, n,  , u, r, l,  , n, ...
11       [w, e, e, k,  , z, l, f,  , f, r,  , n, x, e, ...
12       [h,  , l, r,  , r, e, r,  , h, n,  , n,  , r, ...
13       [v, e, r,  , p, r, n, e,  , e,  , b, e, l, e, ...
14       [h,  , c, h, n, g, e,  , l, f, e,  , p, n, c, ...
15       [h, v, e,  , b, e, e, n,  , k, n, g,  , h,  , ...
16       [r, e,  , e, e, r,  , h, e, l, p,  , w, h,  , .

In [16]:
train['stem']=train['lemma'].apply(stem_words)
test['stem']=test['lemma'].apply(stem_words)

In [32]:
train.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,neg,neu,pos,compound,Cleaned_Text,lemma
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969,0.121,0.879,0.000,-0.2960,has side effect take bystolic and fish oil,"[h, , e, , e, f, f, e, c, , k, e, , b, l, ..."
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458,0.018,0.874,0.108,0.9174,son halfway through his fourth week intuniv be...,"[n, , h, l, f, w, , h, r, u, g, h, , h, , ..."
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969,0.059,0.861,0.080,0.6160,used take another oral which had pill cycle an...,"[u, e, , k, e, , n, h, e, r, , r, l, , w, ..."
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176,0.061,0.771,0.168,0.9403,suboxone has turned life around feel healthier...,"[u, b, x, n, e, , h, , u, r, n, e, , l, f, ..."
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782,0.167,0.754,0.079,-0.4342,day started work with rock hard erections howe...,"[ , r, e, , w, r, k, , w, h, , r, c, k, , ..."


In [33]:
new = train['drug_approved_by_UIC'].str.split("-", n = 2, expand = True) 
train['Day'] = new[0]
train['Month'] = new[1]
train['Year'] = new[2]
del new

In [34]:
new = test['drug_approved_by_UIC'].str.split("-", n = 2, expand = True) 
test['Day'] = new[0]
test['Month'] = new[1]
test['Year'] = new[2]
del new

In [35]:
train['Drug'] = train['name_of_drug']
train['Drug'] = train['Drug'].str.split('/').str[0]

In [36]:
test['Drug'] = test['name_of_drug']
test['Drug'] = test['Drug'].str.split('/').str[0]

In [37]:
train.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug'],
      dtype='object')

In [38]:
test.columns

Index(['patient_id', 'name_of_drug', 'review_by_patient',
       'drug_approved_by_UIC', 'number_of_times_prescribed',
       'use_case_for_drug', 'effectiveness_rating', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug'],
      dtype='object')

In [39]:
Train_Drugs_Eff_Rating = pd.DataFrame()
Train_Drugs_Eff_Rating['Drug'] = train['Drug']
Train_Drugs_Eff_Rating['Year'] = train['Year']
Train_Drugs_Eff_Rating['Avg_Rating'] = train['effectiveness_rating']

In [40]:
Test_Drugs_Eff_Rating = pd.DataFrame()
Test_Drugs_Eff_Rating['Drug'] = test['Drug']
Test_Drugs_Eff_Rating['Year'] = test['Year']
Test_Drugs_Eff_Rating['Avg_Rating'] = test['effectiveness_rating']

In [41]:
Train_Drugs_Eff_Rating = Train_Drugs_Eff_Rating.groupby(['Drug', 'Year'])['Avg_Rating'].mean().round(2)
Train_Drugs_Eff_Rating = Train_Drugs_Eff_Rating.reset_index()

In [42]:
Test_Drugs_Eff_Rating = Test_Drugs_Eff_Rating.groupby(['Drug', 'Year'])['Avg_Rating'].mean().round(2)
Test_Drugs_Eff_Rating = Test_Drugs_Eff_Rating.reset_index()

In [43]:
train = pd.merge(train, Train_Drugs_Eff_Rating, on = (['Drug', 'Year']), how = 'left')

In [44]:
test = pd.merge(test, Test_Drugs_Eff_Rating, on = (['Drug', 'Year']), how = 'left')

In [45]:
Test_Patient_Id = test['patient_id']

In [46]:
train.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug',
       'Avg_Rating'],
      dtype='object')

In [47]:
test.columns

Index(['patient_id', 'name_of_drug', 'review_by_patient',
       'drug_approved_by_UIC', 'number_of_times_prescribed',
       'use_case_for_drug', 'effectiveness_rating', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug',
       'Avg_Rating'],
      dtype='object')

In [48]:
train.drop(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
            'drug_approved_by_UIC', 'Drug'], axis =1, inplace = True)
test.drop(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
           'drug_approved_by_UIC', 'Drug'], axis =1, inplace = True)

In [49]:
train.columns

Index(['effectiveness_rating', 'number_of_times_prescribed', 'base_score',
       'neg', 'neu', 'pos', 'compound', 'Cleaned_Text', 'lemma', 'Day',
       'Month', 'Year', 'Avg_Rating'],
      dtype='object')

In [50]:
test.columns

Index(['number_of_times_prescribed', 'effectiveness_rating', 'neg', 'neu',
       'pos', 'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year',
       'Avg_Rating'],
      dtype='object')

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern='(?u)\\b\\w\\w+\\b', 
                             ngram_range=(1,2), max_df=0.9, min_df=0.01, max_features=None, 
                             binary=False, norm='l2', use_idf=1, smooth_idf=1, sublinear_tf=False)

In [58]:
full_text = list(train['lemma'].values) + list(test['lemma'].values)
tfidf_vect.fit(full_text)

AttributeError: 'list' object has no attribute 'lower'

In [29]:
train_tfidf = tfidf_vect.transform(train['Cleaned_Text'])
test_tfidf = tfidf_vect.transform(test['Cleaned_Text'])

In [30]:
train_tfidf.shape, train.shape

((32165, 549), (32165, 12))

In [31]:
train_df = pd.DataFrame(train_tfidf.toarray(),columns=tfidf_vect.get_feature_names()) 

In [32]:
test_df = pd.DataFrame(test_tfidf.toarray(),columns=tfidf_vect.get_feature_names())

In [33]:
target = train['base_score']

In [34]:
from sklearn.metrics import mean_squared_error
from math import sqrt
#rmse = sqrt(mean_squared_error(y_actual, y_predicted))
def rmsle(y_true, y_pred):
    return 100*max(0,1-sqrt(mean_squared_error(y_true, y_pred)))

In [35]:
train_df['effectiveness_rating'] = train['effectiveness_rating']
train_df['number_of_times_prescribed'] = train['number_of_times_prescribed']
train_df['Pos_Score'] = train['pos']
train_df['Neg_Score'] = train['neg']
train_df['Neu_Score'] = train['neu']
train_df['compound'] = train['compound']
train_df['Avg_Rating'] = train['Avg_Rating']

In [36]:
test_df['effectiveness_rating'] = test['effectiveness_rating']
test_df['number_of_times_prescribed'] = test['number_of_times_prescribed']
test_df['Pos_Score'] = test['pos']
test_df['Neg_Score'] = test['neg']
test_df['Neu_Score'] = test['neu']
test_df['compound'] = test['compound']
test_df['Avg_Rating'] = test['Avg_Rating']

In [37]:
train_df.shape, test_df.shape

((32165, 556), (10760, 556))

In [38]:
#https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(train_df, target, test_size=0.20, random_state=1)

In [40]:
X_train.shape, X_cv.shape

((25732, 556), (6433, 556))

LightGBM Model

In [41]:
import lightgbm as lgb
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_cv, y_cv, reference=lgb_train)

In [42]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 22,
    'sub_feature':{0.5,0.6},
    'max_depth':40,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

In [47]:
gbm = lgb.train(params,
                lgb_train,     # score 85.06
                num_boost_round=15000,
                valid_sets=lgb_eval,
                early_stopping_rounds=250)

[1]	valid_0's l2: 2.24111	valid_0's l1: 1.16834
Training until validation scores don't improve for 250 rounds
[2]	valid_0's l2: 2.00074	valid_0's l1: 1.10876
[3]	valid_0's l2: 1.79109	valid_0's l1: 1.05653
[4]	valid_0's l2: 1.62349	valid_0's l1: 1.00953
[5]	valid_0's l2: 1.46387	valid_0's l1: 0.961928
[6]	valid_0's l2: 1.36476	valid_0's l1: 0.935321
[7]	valid_0's l2: 1.28426	valid_0's l1: 0.914492
[8]	valid_0's l2: 1.21811	valid_0's l1: 0.897203
[9]	valid_0's l2: 1.1152	valid_0's l1: 0.858457
[10]	valid_0's l2: 1.02977	valid_0's l1: 0.823716
[11]	valid_0's l2: 0.93675	valid_0's l1: 0.776897
[12]	valid_0's l2: 0.869553	valid_0's l1: 0.747255
[13]	valid_0's l2: 0.865209	valid_0's l1: 0.746062
[14]	valid_0's l2: 0.809187	valid_0's l1: 0.72196
[15]	valid_0's l2: 0.757066	valid_0's l1: 0.697511
[16]	valid_0's l2: 0.710739	valid_0's l1: 0.673151
[17]	valid_0's l2: 0.673793	valid_0's l1: 0.657096
[18]	valid_0's l2: 0.639625	valid_0's l1: 0.636691
[19]	valid_0's l2: 0.602916	valid_0's l1: 0.61

[159]	valid_0's l2: 0.0488514	valid_0's l1: 0.128104
[160]	valid_0's l2: 0.0489394	valid_0's l1: 0.128048
[161]	valid_0's l2: 0.0482682	valid_0's l1: 0.12702
[162]	valid_0's l2: 0.0481119	valid_0's l1: 0.126498
[163]	valid_0's l2: 0.0480112	valid_0's l1: 0.126303
[164]	valid_0's l2: 0.0475252	valid_0's l1: 0.125376
[165]	valid_0's l2: 0.0474314	valid_0's l1: 0.12503
[166]	valid_0's l2: 0.0470972	valid_0's l1: 0.124542
[167]	valid_0's l2: 0.0469199	valid_0's l1: 0.124157
[168]	valid_0's l2: 0.0467802	valid_0's l1: 0.124105
[169]	valid_0's l2: 0.0465954	valid_0's l1: 0.12373
[170]	valid_0's l2: 0.046231	valid_0's l1: 0.123397
[171]	valid_0's l2: 0.046025	valid_0's l1: 0.123081
[172]	valid_0's l2: 0.0459483	valid_0's l1: 0.122942
[173]	valid_0's l2: 0.0458384	valid_0's l1: 0.122829
[174]	valid_0's l2: 0.0454062	valid_0's l1: 0.121861
[175]	valid_0's l2: 0.0454313	valid_0's l1: 0.121922
[176]	valid_0's l2: 0.0453967	valid_0's l1: 0.121871
[177]	valid_0's l2: 0.0453634	valid_0's l1: 0.12177

[317]	valid_0's l2: 0.035847	valid_0's l1: 0.102333
[318]	valid_0's l2: 0.035845	valid_0's l1: 0.10234
[319]	valid_0's l2: 0.0356382	valid_0's l1: 0.102132
[320]	valid_0's l2: 0.0356421	valid_0's l1: 0.102134
[321]	valid_0's l2: 0.035587	valid_0's l1: 0.102025
[322]	valid_0's l2: 0.0355513	valid_0's l1: 0.101962
[323]	valid_0's l2: 0.0355416	valid_0's l1: 0.101924
[324]	valid_0's l2: 0.0355661	valid_0's l1: 0.101964
[325]	valid_0's l2: 0.0355987	valid_0's l1: 0.101976
[326]	valid_0's l2: 0.0355894	valid_0's l1: 0.101952
[327]	valid_0's l2: 0.0356003	valid_0's l1: 0.101967
[328]	valid_0's l2: 0.0355455	valid_0's l1: 0.101896
[329]	valid_0's l2: 0.0355568	valid_0's l1: 0.10191
[330]	valid_0's l2: 0.035552	valid_0's l1: 0.101879
[331]	valid_0's l2: 0.0356013	valid_0's l1: 0.101919
[332]	valid_0's l2: 0.0355794	valid_0's l1: 0.101895
[333]	valid_0's l2: 0.0355974	valid_0's l1: 0.101936
[334]	valid_0's l2: 0.035533	valid_0's l1: 0.101902
[335]	valid_0's l2: 0.0355289	valid_0's l1: 0.101901


[473]	valid_0's l2: 0.0330164	valid_0's l1: 0.0952779
[474]	valid_0's l2: 0.0330173	valid_0's l1: 0.0952889
[475]	valid_0's l2: 0.033017	valid_0's l1: 0.0952829
[476]	valid_0's l2: 0.0330008	valid_0's l1: 0.0952671
[477]	valid_0's l2: 0.0329838	valid_0's l1: 0.0952405
[478]	valid_0's l2: 0.0330038	valid_0's l1: 0.0952211
[479]	valid_0's l2: 0.0330429	valid_0's l1: 0.0952255
[480]	valid_0's l2: 0.0330318	valid_0's l1: 0.0951729
[481]	valid_0's l2: 0.0330213	valid_0's l1: 0.0951653
[482]	valid_0's l2: 0.0330204	valid_0's l1: 0.0951914
[483]	valid_0's l2: 0.033026	valid_0's l1: 0.095212
[484]	valid_0's l2: 0.033028	valid_0's l1: 0.0952244
[485]	valid_0's l2: 0.0330253	valid_0's l1: 0.0952133
[486]	valid_0's l2: 0.0330161	valid_0's l1: 0.0951646
[487]	valid_0's l2: 0.0330065	valid_0's l1: 0.0951525
[488]	valid_0's l2: 0.0329968	valid_0's l1: 0.095153
[489]	valid_0's l2: 0.0329892	valid_0's l1: 0.0951497
[490]	valid_0's l2: 0.0329581	valid_0's l1: 0.0950421
[491]	valid_0's l2: 0.0329523	val

[627]	valid_0's l2: 0.032075	valid_0's l1: 0.0923099
[628]	valid_0's l2: 0.0320701	valid_0's l1: 0.092289
[629]	valid_0's l2: 0.0320653	valid_0's l1: 0.0922732
[630]	valid_0's l2: 0.0320579	valid_0's l1: 0.0922588
[631]	valid_0's l2: 0.0320595	valid_0's l1: 0.092251
[632]	valid_0's l2: 0.03206	valid_0's l1: 0.0922428
[633]	valid_0's l2: 0.0320629	valid_0's l1: 0.0922411
[634]	valid_0's l2: 0.0320555	valid_0's l1: 0.0922209
[635]	valid_0's l2: 0.0320539	valid_0's l1: 0.0922301
[636]	valid_0's l2: 0.0320536	valid_0's l1: 0.0922372
[637]	valid_0's l2: 0.0320545	valid_0's l1: 0.0922433
[638]	valid_0's l2: 0.0320485	valid_0's l1: 0.0922484
[639]	valid_0's l2: 0.0320483	valid_0's l1: 0.0922472
[640]	valid_0's l2: 0.0320456	valid_0's l1: 0.0922522
[641]	valid_0's l2: 0.032051	valid_0's l1: 0.0922522
[642]	valid_0's l2: 0.0320521	valid_0's l1: 0.0922574
[643]	valid_0's l2: 0.0320646	valid_0's l1: 0.0922566
[644]	valid_0's l2: 0.0320669	valid_0's l1: 0.0922653
[645]	valid_0's l2: 0.0320583	vali

...

[3051]	valid_0's l2: 0.0290092	valid_0's l1: 0.0823506
[3052]	valid_0's l2: 0.029008	valid_0's l1: 0.0823509
[3053]	valid_0's l2: 0.0290081	valid_0's l1: 0.0823502
[3054]	valid_0's l2: 0.0290091	valid_0's l1: 0.0823469
[3055]	valid_0's l2: 0.0290092	valid_0's l1: 0.0823445
[3056]	valid_0's l2: 0.0290088	valid_0's l1: 0.0823419
[3057]	valid_0's l2: 0.0290086	valid_0's l1: 0.0823441
[3058]	valid_0's l2: 0.0290088	valid_0's l1: 0.0823426
[3059]	valid_0's l2: 0.0290086	valid_0's l1: 0.0823434
[3060]	valid_0's l2: 0.0290089	valid_0's l1: 0.0823494
[3061]	valid_0's l2: 0.0290088	valid_0's l1: 0.0823487
[3062]	valid_0's l2: 0.0290094	valid_0's l1: 0.0823459
[3063]	valid_0's l2: 0.0290093	valid_0's l1: 0.0823396
[3064]	valid_0's l2: 0.0290089	valid_0's l1: 0.0823393
[3065]	valid_0's l2: 0.0290094	valid_0's l1: 0.0823383
[3066]	valid_0's l2: 0.0290081	valid_0's l1: 0.0823367
[3067]	valid_0's l2: 0.0290077	valid_0's l1: 0.0823362
[3068]	valid_0's l2: 0.0290082	valid_0's l1: 0.0823423
[3069]	vali

[3203]	valid_0's l2: 0.0289595	valid_0's l1: 0.0821984
[3204]	valid_0's l2: 0.0289599	valid_0's l1: 0.0822041
[3205]	valid_0's l2: 0.0289599	valid_0's l1: 0.0822008
[3206]	valid_0's l2: 0.0289612	valid_0's l1: 0.0822003
[3207]	valid_0's l2: 0.0289607	valid_0's l1: 0.0821964
[3208]	valid_0's l2: 0.0289597	valid_0's l1: 0.0821989
[3209]	valid_0's l2: 0.0289602	valid_0's l1: 0.0821994
[3210]	valid_0's l2: 0.0289622	valid_0's l1: 0.0821984
[3211]	valid_0's l2: 0.0289644	valid_0's l1: 0.0821998
[3212]	valid_0's l2: 0.028963	valid_0's l1: 0.0821962
[3213]	valid_0's l2: 0.0289623	valid_0's l1: 0.0821929
[3214]	valid_0's l2: 0.0289628	valid_0's l1: 0.0821965
[3215]	valid_0's l2: 0.0289632	valid_0's l1: 0.0821977
[3216]	valid_0's l2: 0.0289523	valid_0's l1: 0.0821447
[3217]	valid_0's l2: 0.0289513	valid_0's l1: 0.08214
[3218]	valid_0's l2: 0.0289512	valid_0's l1: 0.0821422
[3219]	valid_0's l2: 0.0289502	valid_0's l1: 0.0821433
[3220]	valid_0's l2: 0.0289496	valid_0's l1: 0.0821398
[3221]	valid_

[3355]	valid_0's l2: 0.028935	valid_0's l1: 0.0820925
[3356]	valid_0's l2: 0.028935	valid_0's l1: 0.0820924
[3357]	valid_0's l2: 0.0289354	valid_0's l1: 0.0820923
[3358]	valid_0's l2: 0.0289346	valid_0's l1: 0.0820939
[3359]	valid_0's l2: 0.0289335	valid_0's l1: 0.0820893
[3360]	valid_0's l2: 0.0289326	valid_0's l1: 0.0820888
[3361]	valid_0's l2: 0.0289327	valid_0's l1: 0.0820917
[3362]	valid_0's l2: 0.0289317	valid_0's l1: 0.0820896
[3363]	valid_0's l2: 0.028932	valid_0's l1: 0.0820879
[3364]	valid_0's l2: 0.0289316	valid_0's l1: 0.082088
[3365]	valid_0's l2: 0.028933	valid_0's l1: 0.0820878
[3366]	valid_0's l2: 0.028929	valid_0's l1: 0.0820724
[3367]	valid_0's l2: 0.0289308	valid_0's l1: 0.0820708
[3368]	valid_0's l2: 0.028923	valid_0's l1: 0.0820552
[3369]	valid_0's l2: 0.0289197	valid_0's l1: 0.0820463
[3370]	valid_0's l2: 0.0289199	valid_0's l1: 0.0820459
[3371]	valid_0's l2: 0.0289192	valid_0's l1: 0.0820428
[3372]	valid_0's l2: 0.0289187	valid_0's l1: 0.0820453
[3373]	valid_0's 

[3505]	valid_0's l2: 0.0289203	valid_0's l1: 0.0819761
[3506]	valid_0's l2: 0.0289193	valid_0's l1: 0.0819749
[3507]	valid_0's l2: 0.0289176	valid_0's l1: 0.0819718
[3508]	valid_0's l2: 0.0289189	valid_0's l1: 0.0819703
[3509]	valid_0's l2: 0.0289203	valid_0's l1: 0.0819635
[3510]	valid_0's l2: 0.0289201	valid_0's l1: 0.0819689
[3511]	valid_0's l2: 0.0289205	valid_0's l1: 0.0819715
[3512]	valid_0's l2: 0.0289224	valid_0's l1: 0.0819732
[3513]	valid_0's l2: 0.0289229	valid_0's l1: 0.0819751
[3514]	valid_0's l2: 0.0289235	valid_0's l1: 0.0819749
[3515]	valid_0's l2: 0.0289244	valid_0's l1: 0.0819745
[3516]	valid_0's l2: 0.0289252	valid_0's l1: 0.0819757
[3517]	valid_0's l2: 0.0289258	valid_0's l1: 0.0819776
[3518]	valid_0's l2: 0.0289243	valid_0's l1: 0.0819737
[3519]	valid_0's l2: 0.0289245	valid_0's l1: 0.0819765
[3520]	valid_0's l2: 0.0289248	valid_0's l1: 0.0819779
[3521]	valid_0's l2: 0.0289252	valid_0's l1: 0.0819779
[3522]	valid_0's l2: 0.028925	valid_0's l1: 0.081977
[3523]	valid

[3655]	valid_0's l2: 0.0289413	valid_0's l1: 0.0820019
[3656]	valid_0's l2: 0.0289387	valid_0's l1: 0.0820025
[3657]	valid_0's l2: 0.0289398	valid_0's l1: 0.0820017
[3658]	valid_0's l2: 0.0289389	valid_0's l1: 0.0820026
[3659]	valid_0's l2: 0.0289388	valid_0's l1: 0.0820024
[3660]	valid_0's l2: 0.0289337	valid_0's l1: 0.0819966
[3661]	valid_0's l2: 0.0289328	valid_0's l1: 0.0819962
[3662]	valid_0's l2: 0.028933	valid_0's l1: 0.0819956
[3663]	valid_0's l2: 0.0289325	valid_0's l1: 0.0819948
[3664]	valid_0's l2: 0.0289335	valid_0's l1: 0.0819957
[3665]	valid_0's l2: 0.0289329	valid_0's l1: 0.0819923
[3666]	valid_0's l2: 0.0289318	valid_0's l1: 0.0819889
[3667]	valid_0's l2: 0.0289311	valid_0's l1: 0.0819868
[3668]	valid_0's l2: 0.0289326	valid_0's l1: 0.0819887
[3669]	valid_0's l2: 0.0289361	valid_0's l1: 0.0819928
[3670]	valid_0's l2: 0.0289362	valid_0's l1: 0.0819946
[3671]	valid_0's l2: 0.0289363	valid_0's l1: 0.0819991
[3672]	valid_0's l2: 0.0289373	valid_0's l1: 0.082001
[3673]	valid

In [48]:
y_pred = gbm.predict(X_cv, num_iteration=gbm.best_iteration)
print('The rmse of prediction is:', rmsle(y_cv, y_pred))

The rmse of prediction is: 82.99518059199076


In [66]:
y_tpred = gbm.predict(X_train, num_iteration=gbm.best_iteration)
print('The rmse of prediction is:', rmsle(y_train, y_tpred))

The rmse of prediction is: 96.35952372461443


In [57]:
Predictions = gbm.predict(test_df, num_iteration=gbm.best_iteration)

In [145]:
import xgboost
xgb = xgboost.XGBRegressor(objective ='reg:linear', n_estimators=1800, learning_rate=0.001, gamma=0, subsample=0.9,
                           colsample_bytree=0.9, max_depth=22)
xgb.fit(X_train,y_train)

C:\Users\hp\Anaconda3\3.7\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:27:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=1,
             importance_type='gain', learning_rate=0.001, max_delta_step=0,
             max_depth=22, min_child_weight=1, missing=None, n_estimators=1800,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=1)

In [146]:
xgb_y_tpred = xgb.predict(X_train)
xgb_y_pred = xgb.predict(X_cv)

In [147]:
print('Train rmse of prediction is:', rmsle(y_train, xgb_y_tpred))
print('Test rmse of prediction is:', rmsle(y_cv, xgb_y_pred))

Train rmse of prediction is: 0
Test rmse of prediction is: 0


In [148]:
Predictions_XGB = xgb.predict(test_df)

Gradient Boosting

In [157]:
from sklearn.ensemble import GradientBoostingRegressor
GBoost = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,
                                   max_depth=30,
                                   min_samples_leaf=15, min_samples_split=15, 
                                   loss='huber', random_state =5)

In [159]:
GBoost.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='huber',
                          max_depth=30, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=15,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=5, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [160]:
gb_y_tpred = GBoost.predict(X_train)
print('The train rmse of prediction is:', rmsle(y_train, gb_y_tpred))

The train rmse of prediction is: 89.50134893505512


In [161]:
gb_y_pred = GBoost.predict(X_cv)
print('The test rmse of prediction is:', rmsle(y_cv, gb_y_pred))

The test rmse of prediction is: 74.82774009251982


In [162]:
Predictions_GBM = GBoost.predict(test_df)

In [251]:
ens_train = y_tpred*0.50 + gb_y_tpred*0.5

In [267]:
ensemble =  Predictions*0.9 + Predictions_GBM*0.1

In [268]:
print('The train rmse of prediction is:', rmsle(y_train, ens_train))

The train rmse of prediction is: 86.55602782667182


In [58]:
Submission = pd.DataFrame()
Submission['patient_id'] = Test_Patient_Id
Submission['base_score'] = Predictions

In [59]:
Submission.to_csv('STD_Preds.csv', index=False)

In [102]:
Submission.shape, test_df.shape

((10760, 2), (10760, 603))

In [56]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [57]:
rf_cv_pred = rf.predict(X_cv)
print('The test rmse of prediction is:', rmsle(y_cv,rf_cv_pred))

The test rmse of prediction is: 0


In [55]:
rf_y_tpred = rf.predict(X_train)
print('The train rmse of prediction is:', rmsle(y_train, rf_y_tpred))

The train rmse of prediction is: 88.19708905079683


In [59]:
from sklearn.linear_model import BayesianRidge
br=BayesianRidge()
br.fit(X_train,y_train)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [60]:
br_cv_pred = br.predict(X_cv)
print('The test rmse of prediction is:', rmsle(y_cv,br_cv_pred))

The test rmse of prediction is: 0
